## 'Dogs vs Cats' in Keras/Tensorflow

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from keras import backend as K

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
K.set_session(sess)

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout, Flatten, Dense
from keras.applications import ResNet50
from keras.models import Model, Sequential
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.applications.resnet50 import preprocess_input
from keras.optimizers import Adam

In [ ]:
PATH = "../data/dogscats/"
sz,bs=224,64
train_data_dir,validation_data_dir = f'{PATH}train',f'{PATH}valid'

In [ ]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
    shear_range=0.2, zoom_range=0.2, rotation_range=10, brightness_range=(0.8,1.2), horizontal_flip=True)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
    target_size=(sz, sz), batch_size=bs, class_mode='binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
    target_size=(sz, sz), shuffle=False,
    batch_size=bs, class_mode='binary')

In [ ]:
base_model = ResNet50(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

In [ ]:
lr,num_workers = 1e-3,16

model = Model(inputs=base_model.input, outputs=predictions)
opt = Adam(lr=lr)
for layer in base_model.layers: layer.trainable = False
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
model.fit_generator(train_generator, train_generator.n // bs, epochs=6, workers=num_workers,
        validation_data=validation_generator, validation_steps=validation_generator.n // bs)

In [ ]:
model.save_weights(f'{PATH}/models/keras_0.h5')

In [ ]:
model.load_weights(f'{PATH}/models/keras_0.h5')

In [ ]:
for layer in model.layers: layer.trainable = True
K.set_value(opt.lr , lr/100)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
%%time
model.fit_generator(train_generator, train_generator.n // bs, epochs=6, workers=num_workers,
        validation_data=validation_generator, validation_steps=validation_generator.n // bs)